In [4]:
# Packages
import pynance as pn
import pandas_datareader.data as web
import pandas as pd
import gc 
from datetime import datetime
from datetime import timedelta
import numpy as np

# Directories
transform = '../../data/transform'


# del my_array
# del my_object
# gc.collect()

# Import a list of stocks

In [5]:
snp_companies = pd.read_csv(f'{transform}/dim_snp_esg_full_16-Jan-2022.csv')

In [6]:
snp_companies

,ticker,company_name,gics_sector,gics_sub_industry,is_esg
0,A,Agilent Technologies,Health Care,Health Care Equipment,1
1,AAL,American Airlines Group,Industrials,Airlines,1
2,AAP,Advance Auto Parts,Consumer Discretionary,Automotive Retail,0
3,AAPL,Apple,Information Technology,"Technology Hardware, Storage & Peripherals",1
4,ABBV,AbbVie,Health Care,Pharmaceuticals,1
...,...,...,...,...,...
502,YUM,Yum! Brands,Consumer Discretionary,Restaurants,1
503,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,0
504,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,1
505,ZION,Zions Bancorp,Financials,Regional Banks,0


In [7]:
# Filter for ESG companies and the benchmarks
snp_companies_esg = snp_companies[snp_companies['is_esg']==0]
snp_companies_etf = snp_companies[snp_companies['gics_sector']=='ETF']
snp_companies_esg_etf = snp_companies_esg.append(snp_companies_etf)

# Data cleaning
snp_companies_esg_etf['ticker'] = snp_companies_esg_etf['ticker'].str.replace(r'.', '-',regex=True)
snp_companies_esg_etf = snp_companies_esg_etf.drop_duplicates()

## Preparing the dataframe to store daily stock prices

In [8]:
stock_prices = pd.DataFrame()

# Seperated from stock price query because the date range in getting stock price will be converted to incremental loading
# This section is akin to backfilling
start_date = '2021-01-01' 
end_date = '2022-01-14'

# This is done because Yahoo strangely takes your intended query date - 1 day. Hence the base table should match this
base_start_date = (datetime.strptime(start_date, "%Y-%m-%d") - timedelta(days=1)).strftime(format="%Y-%m-%d")
base_end_date = end_date

# This behaves as an index
stock_prices['date'] = pd.Series(pd.date_range(start=base_start_date, end=base_end_date,freq='D'))

# Business Days
biz_date = pd.DataFrame(pd.bdate_range(start=base_start_date, end=base_end_date,freq='B'), columns=['biz_date'])

# Join dates and biz_date to detect weekends or hidden holidays
stock_prices = stock_prices.merge(biz_date,
                                  how='left',
                                  left_on='date',
                                  right_on='biz_date')

# The day of the week with Monday=0, Sunday=6.
stock_prices['day'] = stock_prices['date'].dt.dayofweek

## Retrieving Stock Prices

In [9]:
# Need to add data quality check here to ensure dates do not go out of range

In [10]:
# Column checker
stock_name_list = snp_companies_esg_etf.ticker.to_list()

print(f"""
We expect an additional {len(stock_name_list)} columns when the query is done

There are {stock_prices.shape[1]} columns with shape as {stock_prices.shape} in the base dataframe with column names as : {stock_prices.columns.format()}

Therefore, when the data is loaded, there should be {len(stock_name_list) + stock_prices.shape[1]} columns
""")


We expect an additional 197 columns when the query is done

There are 3 columns with shape as (380, 3) in the base dataframe with column names as : ['date', 'biz_date', 'day']

Therefore, when the data is loaded, there should be 200 columns



In [11]:
count = 1
error_tickers = []
for stock in stock_name_list:
    try:
        temp_stock_info_df = pd.DataFrame()

        print(f'Processing {stock}')
        print(f'Stock {count} of {len(stock_name_list)}')

        # temp_stock_info_df = (pd
        #                       .DataFrame(
        #                           pn.data.get(stock, base_start_date, base_end_date).loc[:,'Close'])
        #                       .rename({'Close':f'{stock}'},axis=1)
        #                      )
        
        temp_stock_info_df = (pd.DataFrame((web.DataReader(stock, 'yahoo', start=start_date, end=end_date)
                                               .loc[:,'Close']))
                                .rename({'Close':f'{stock}'},axis=1))

        print(f'Successfully processed {stock} at {datetime.today()}!')

        stock_prices = (stock_prices.merge(temp_stock_info_df, 
                                           how='left', 
                                           left_on='date', 
                                           right_on='Date'))

        print(f'DataFrame size is {stock_prices.shape}.\n')

        count += 1

        # Reduce RAM consumption
        del temp_stock_info_df
        gc.collect()
    
    except KeyError:
        error_tickers.append(stock)
        print(f'This ticker {stock} is not found.\n')
        continue

Processing AAP
Stock 1 of 197
Successfully processed AAP at 2022-01-17 21:31:47.615778!
DataFrame size is (380, 4).

Processing ABMD
Stock 2 of 197
Successfully processed ABMD at 2022-01-17 21:31:48.576256!
DataFrame size is (380, 5).

Processing ADP
Stock 3 of 197
Successfully processed ADP at 2022-01-17 21:31:49.555369!
DataFrame size is (380, 6).

Processing AEE
Stock 4 of 197
Successfully processed AEE at 2022-01-17 21:31:51.049876!
DataFrame size is (380, 7).

Processing AEP
Stock 5 of 197
Successfully processed AEP at 2022-01-17 21:31:52.109272!
DataFrame size is (380, 8).

Processing AES
Stock 6 of 197
Successfully processed AES at 2022-01-17 21:31:53.071469!
DataFrame size is (380, 9).

Processing AIG
Stock 7 of 197
Successfully processed AIG at 2022-01-17 21:31:54.003405!
DataFrame size is (380, 10).

Processing ALGN
Stock 8 of 197
Successfully processed ALGN at 2022-01-17 21:31:55.014319!
DataFrame size is (380, 11).

Processing ALLE
Stock 9 of 197
Successfully processed ALLE

Successfully processed FBHS at 2022-01-17 21:32:59.697343!
DataFrame size is (380, 73).

Processing FDS
Stock 71 of 197
Successfully processed FDS at 2022-01-17 21:33:00.682119!
DataFrame size is (380, 74).

Processing FFIV
Stock 72 of 197
Successfully processed FFIV at 2022-01-17 21:33:02.288073!
DataFrame size is (380, 75).

Processing FIS
Stock 73 of 197
Successfully processed FIS at 2022-01-17 21:33:03.256545!
DataFrame size is (380, 76).

Processing FISV
Stock 74 of 197
Successfully processed FISV at 2022-01-17 21:33:04.425735!
DataFrame size is (380, 77).

Processing FLT
Stock 75 of 197
Successfully processed FLT at 2022-01-17 21:33:05.394637!
DataFrame size is (380, 78).

Processing FOX
Stock 76 of 197
Successfully processed FOX at 2022-01-17 21:33:06.152609!
DataFrame size is (380, 79).

Processing FOXA
Stock 77 of 197
Successfully processed FOXA at 2022-01-17 21:33:07.154962!
DataFrame size is (380, 80).

Processing FRC
Stock 78 of 197
Successfully processed FRC at 2022-01-17 

Successfully processed PKG at 2022-01-17 21:34:14.147671!
DataFrame size is (380, 141).

Processing PM
Stock 139 of 197
Successfully processed PM at 2022-01-17 21:34:15.339098!
DataFrame size is (380, 142).

Processing PNW
Stock 140 of 197
Successfully processed PNW at 2022-01-17 21:34:16.933081!
DataFrame size is (380, 143).

Processing POOL
Stock 141 of 197
Successfully processed POOL at 2022-01-17 21:34:17.917293!
DataFrame size is (380, 144).

Processing PPL
Stock 142 of 197
Successfully processed PPL at 2022-01-17 21:34:18.893805!
DataFrame size is (380, 145).

Processing PSA
Stock 143 of 197
Successfully processed PSA at 2022-01-17 21:34:19.927019!
DataFrame size is (380, 146).

Processing PSX
Stock 144 of 197
Successfully processed PSX at 2022-01-17 21:34:20.931713!
DataFrame size is (380, 147).

Processing PTC
Stock 145 of 197
Successfully processed PTC at 2022-01-17 21:34:21.928526!
DataFrame size is (380, 148).

Processing PWR
Stock 146 of 197
Successfully processed PWR at 20

In [12]:
stock_prices.head()

,date,biz_date,day,AAP,ABMD,ADP,AEE,AEP,AES,AIG,...,WRB,WTW,WYNN,XEL,XLNX,XRAY,ZBH,ZION,ZTS,SNPE
0,2020-12-31,2020-12-31,3,157.509995,324.200012,176.199997,78.059998,83.269997,23.50,37.860001,...,66.419998,210.679993,112.830002,66.669998,141.770004,52.360001,154.089996,43.439999,165.500000,32.840000
1,2021-01-01,2021-01-01,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-02,NaT,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-03,NaT,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-04,2021-01-04,0,157.339996,316.730011,168.960007,76.129997,81.540001,23.24,37.130001,...,64.940002,203.699997,106.900002,65.660004,142.429993,53.750000,153.160004,42.930000,163.589996,32.369999


In [16]:
stock_prices_long = pd.melt(stock_prices, 
                            id_vars=['date', 'biz_date', 'day'],
                            value_vars=stock_prices.columns[3:],
                            var_name='ticker',
                            value_name='price')

In [17]:
stock_prices_long

,date,biz_date,day,ticker,price
0,2020-12-31,2020-12-31,3,AAP,157.509995
1,2021-01-01,2021-01-01,4,AAP,NaN
2,2021-01-02,NaT,5,AAP,NaN
3,2021-01-03,NaT,6,AAP,NaN
4,2021-01-04,2021-01-04,0,AAP,157.339996
...,...,...,...,...,...
74855,2022-01-10,2022-01-10,0,SNPE,41.810001
74856,2022-01-11,2022-01-11,1,SNPE,42.200001
74857,2022-01-12,2022-01-12,2,SNPE,42.349998
74858,2022-01-13,2022-01-13,3,SNPE,41.730000


In [18]:
stock_prices.to_csv(f"{transform_path}/non_esg_stock_prices_wide_{datetime.now().date()}.csv",index=False)
stock_prices_long.to_csv(f"{transform_path}/non_esg_stock_prices_long_{datetime.now().date()}.csv",index=False)

In [19]:
print(error_tickers)

[]
